In [4]:
'''Connecting to DB'''
import pyodbc
Server='qa-datagov01.nexjqa.local'
DB='PRO_MAJIK_CARMAX'
Ver='sql2012'
user='nexj'
password='nexj'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+Server+';DATABASE='+DB+';UID='+user+';PWD='+password)
# Create a cursor from the connection
cursor = cnxn.cursor()

In [9]:
#TODO:  NEEDS TO CHANGE 

#first we need to read all of the error codes
sqlStr=' SELECT DISTINCT [errorCode]     FROM [PRO_MAJIK_CARMAX].[dbo].[\'elinedt-Majik_full-data-set_pre$\']'

cursor.execute(sqlStr)    
buffers = cursor.fetchall()

errorcodes=[]
for buffer in buffers: 
    errorcodes.append (buffer[0])#For once we have type , subtype((buffer[0],buffer[1]))  #TODO : make sure buffer is right
# errorCodeprint (errorcodes)  
# Read the equipment 
# 



#Since Equipments at the monemt don't exist we ignore it here 
# sqlStr='SELECT [id]  FROM [RPO_MAJIK_DISCOVERY].[dbo].[equipment]'  
# cursor.execute(sqlStr)    
# buffers = cursor.fetchall()
# equipments=[]
# # eqStr="("

# for buffer in buffers: 
#     equipments.append (buffer[0]) 
# #     eqStr+='[E'+buffer[0].replace("-", "0")+']'# becuase thats how we name the columns in SQL 
# #     eqStr+=','


# # eqStr+="[errorID])"
# # #print (eqStr)
# eqStr="([ErrorId],[line],[ErrorType],[DeviceId],[MTTF],[MeanDownT])"


In [12]:
'''Calculating the mean time to failure(MTTF)
Updating and adding the new data 
DO not run this and the next box at the same time
'''
from datetime import datetime

c=0
eqStr="([ErrorId],[line],[ErrorType],[DeviceId],[TFsum],[DownTimeSum],[Fnum])" 
for error in errorcodes:
    if error==None:
        error="N/A"
        
    c+=1
#     if c>1:
#         break
    sqlStr=' select * FROM [PRO_MAJIK_CARMAX].[dbo].[\'elinedt-Majik_full-data-set_pre$\'] WHERE [errorCode]=LTRIM(RTRIM(\''+error+'\')) order by start'
    cursor.execute(sqlStr)    
    '''Read all at once '''
    rows = cursor.fetchall()
    prev=0  # previouse value
    avgMTTF=0
    avgDown=0
    count=0
    line=1 #For now We have line 1 
    
#     for row in rows:
# #         print(row)
    strWrite="" 
    for row in rows:
        if prev==0:
#             prev =datetime.strptime(row[1], '%Y-%m-%d %H:%M')
            prev=row[1]

        startTime=row[1]

        count+=1

        avgMTTF+=(startTime-prev).total_seconds()#/3600   
        avgDown+=row[3] 
        prev=row[2]

#     if(count==0):
#         count=1
    #write back to db ?
    
#     avgMTTF=avgMTTF/count
#     avgDown=avgDown/count        
       



    strWrite+="\'"+error+"\'"+','
    strWrite+=str(line)+','
#         strWrite+="\'"+str(line)+"\'"+','
    strWrite+="\'"+"UNPLANNED"+"\'"+',' #for now error type and id are the same
    strWrite+="\'"+"equipment"+"\'"+','#we don't have a name for it yet
    strWrite+=str(avgMTTF)+','
    strWrite+=str(avgDown)+','
    strWrite+=str(count)


#     strWrite+=")"




    sqlStr=' UPDATE [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl] set [Fnum]='+str(count)+',[TFsum]='+str(avgMTTF)+',[DownTimeSum]='+str(avgDown)+' WHERE  [ErrorId] = \''+error+'\' AND [DeviceId]=\''+"equipment"+'\';  INSERT INTO [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl]'+eqStr+' select '+strWrite+' WHERE NOT EXISTS(select * From [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl]  WHERE  [ErrorId] = \''+error+'\'AND [DeviceId]=\''+"equipment"+'\' );'
    cursor.execute(sqlStr)   
    cnxn.commit()
#     print (sqlStr)
#     break

strWrite=""


           
            


print("Done")






Done


In [3]:
'''Calculating the mean time to failure(MTTF)
JUST ADDING THE NEW DATA , Ignore the old data, however it will recalculate them  
'''
from datetime import datetime

c=0
eqStr="([ErrorId],[line],[ErrorType],[DeviceId],[TFsum],[DownTimeSum],[Fnum])" 
for error in errorcodes:
    if error==None:
        error="N/A"
        
    c+=1
#     if c>1:
#         break
    sqlStr=' select * FROM [PRO_MAJIK_CARMAX].[dbo].[\'elinedt-Majik_full-data-set_pre$\'] WHERE [errorCode]=LTRIM(RTRIM(\''+error+'\')) order by start'
    cursor.execute(sqlStr)    
    '''Read all at once '''
    rows = cursor.fetchall()
    prev=0  # previouse value
    avgMTTF=0
    avgDown=0
    count=0
    line=1 #For now We have line 1 
    
#     for row in rows:
# #         print(row)
    strWrite="" 
    for row in rows:
        if prev==0:
#             prev =datetime.strptime(row[1], '%Y-%m-%d %H:%M')
            prev=row[1]

        startTime=row[1]

        count+=1

        avgMTTF+=(startTime-prev).total_seconds()#/3600   
        avgDown+=row[3] 
        prev=row[2]

    if(count==0):
        count=1
    #write back to db ?
    
#     avgMTTF=avgMTTF/count
#     avgDown=avgDown/count        
       



    strWrite+="\'"+error+"\'"+','
    strWrite+=str(line)+','
#         strWrite+="\'"+str(line)+"\'"+','
    strWrite+="\'"+"UNPLANNED"+"\'"+',' #for now error type and id are the same
    strWrite+="\'"+"equipment"+"\'"+','#we don't have a name for it yet
    strWrite+=str(avgMTTF)+','
    strWrite+=str(avgDown)+','
    strWrite+=str(count)


#     strWrite+=")"




#     strWrite+=")"





    sqlStr=' INSERT INTO [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl]'+eqStr+' select '+strWrite+' WHERE NOT EXISTS(select * From [PRO_MAJIK_CARMAX].[dbo].[MTTFtbl]  WHERE  [ErrorId] = \''+error+'\'AND [DeviceId]=\''+"equipment"+'\' );'

    cursor.execute(sqlStr)   
    cnxn.commit()
#     print (sqlStr)
#     break
strWrite=""


           
            


print("Done")






Done
